<a href="https://colab.research.google.com/github/CygnusST3RN/FHE.0/blob/ankita_ml/FHE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import BFV_Scheme.bfv as bfv
import random


Initialize BFV Parameters

In [2]:

params = bfv.Params(4, 2, 2**27, 2**75)
fv12= bfv.FV12(params)
public_key,private_key=fv12.generate_keys()


Initialised
Eval Key:0/75 Done
Eval Key:1/75 Done
Eval Key:2/75 Done
Eval Key:3/75 Done
Eval Key:4/75 Done
Eval Key:5/75 Done
Eval Key:6/75 Done
Eval Key:7/75 Done
Eval Key:8/75 Done
Eval Key:9/75 Done
Eval Key:10/75 Done
Eval Key:11/75 Done
Eval Key:12/75 Done
Eval Key:13/75 Done
Eval Key:14/75 Done
Eval Key:15/75 Done
Eval Key:16/75 Done
Eval Key:17/75 Done
Eval Key:18/75 Done
Eval Key:19/75 Done
Eval Key:20/75 Done
Eval Key:21/75 Done
Eval Key:22/75 Done
Eval Key:23/75 Done
Eval Key:24/75 Done
Eval Key:25/75 Done
Eval Key:26/75 Done
Eval Key:27/75 Done
Eval Key:28/75 Done
Eval Key:29/75 Done
Eval Key:30/75 Done
Eval Key:31/75 Done
Eval Key:32/75 Done
Eval Key:33/75 Done
Eval Key:34/75 Done
Eval Key:35/75 Done
Eval Key:36/75 Done
Eval Key:37/75 Done
Eval Key:38/75 Done
Eval Key:39/75 Done
Eval Key:40/75 Done
Eval Key:41/75 Done
Eval Key:42/75 Done
Eval Key:43/75 Done
Eval Key:44/75 Done
Eval Key:45/75 Done
Eval Key:46/75 Done
Eval Key:47/75 Done
Eval Key:48/75 Done
Eval Key:49/75 Don

In [3]:
for i in range(10):
    
    a=random.randrange(-10,10)
   
    print('Unencrypted:')
    print('a^2='+str(a**2))
    a_encrypted=public_key.encrypt(a)
   
    print('Encrypted:')
    print('a^2='+str(private_key.decrypt(a_encrypted**2)))

Unencrypted:
a^2=36
Encrypted:


a^2=34
Unencrypted:
a^2=81
Encrypted:
a^2=80
Unencrypted:
a^2=81
Encrypted:
a^2=82
Unencrypted:
a^2=1
Encrypted:
a^2=2
Unencrypted:
a^2=81
Encrypted:
a^2=65
Unencrypted:
a^2=9
Encrypted:
a^2=13
Unencrypted:
a^2=9
Encrypted:
a^2=10
Unencrypted:
a^2=36
Encrypted:
a^2=30
Unencrypted:
a^2=0
Encrypted:
a^2=-2
Unencrypted:
a^2=4
Encrypted:
a^2=-2


In [4]:
a=public_key.encrypt(100)
print(private_key.decrypt((a**2)-a))

9898


Scale Input Values Based on Plaintext modulus and read Dataset 

In [5]:
def read_dataset(url,col,scale):
  df=pd.read_csv(url)
  df.drop("Unnamed: 32", axis=1, inplace=True)
  X = df.iloc[:, col+1:]
  col_name=df.columns[col]
  y = df[col_name]
  for i in range (0,len(df)):
    if y[i]=='M':
      y[i]=1
    else:
      y[i]=0
  X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)
  X_train=np.asarray(X_train).astype('float32')
  X_test=np.asarray(X_test).astype('float32')
  y_train=np.asarray(y_train).astype('float32')
  y_test=np.asarray(y_test).astype('float32')
  scaler = MinMaxScaler(feature_range=(0, scale))
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  print(X_test.shape)
  return X_train, X_test, y_train, y_test


In [6]:
X_train, X_test, y_train, y_test = read_dataset('../dataset/data.csv',1,100)

(143, 30)


C:\Users\delwy\AppData\Local\Temp\ipykernel_16892\3440237989.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=1
C:\Users\delwy\AppData\Local\Temp\ipykernel_16892\3440237989.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=0


Train Model

In [7]:
def custom_relu(x):
    return x +x*x
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(30,)),
    tf.keras.layers.Dense(2),
    # tf.keras.layers.Activation(custom_relu),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=70, batch_size=16)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/70
27/27 [==============================] - 1s 2ms/step - loss: 27.1292 - accuracy: 0.5798
Epoch 2/70
27/27 [==============================] - 0s 2ms/step - loss: 17.5740 - accuracy: 0.4695
Epoch 3/70
27/27 [==============================] - 0s 2ms/step - loss: 13.7878 - accuracy: 0.3873
Epoch 4/70
27/27 [==============================] - 0s 2ms/step - loss: 11.5885 - accuracy: 0.4131
Epoch 5/70
27/27 [==============================] - 0s 2ms/step - loss: 9.7096 - accuracy: 0.4531
Epoch 6/70
27/27 [==============================] - 0s 2ms/step - loss: 8.1004 - accuracy: 0.4883
Epoch 7/70
27/27 [==============================] - 0s 2ms/step - loss: 6.6698 - accuracy: 0.5188
Epoch 8/70
27/27 [==============================] - 0s 2ms/step - loss: 5.4914 - accuracy: 0.5493
Epoch 9/70
27/27 [==============================] - 0s 2ms/step - loss: 4.4874 - accuracy: 0.5915
Epoch 10/70
27/27 [==============================] - 0s 2ms/step - loss: 3.7117 - accuracy: 0.6150
Epoch 11/70
27/

In [8]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 62        
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________
None


Save Quantised Model Weights

In [9]:
def create_model_weights_file(model,scale):
  i=0
  for layer in model.layers:
    if('activation' in layer.name):
      continue
    w = (layer.get_weights()[0]).T
    
    b = (layer.get_weights()[1]).T
    w=np.round_(w*scale).astype(int)
    b=np.round_(b*scale).astype(int)
    file_name = 'layer'+str(i)+'.npy'
    with open(file_name, 'wb') as f:
        np.save(f, w)
        np.save(f, b)
    i=i+1


In [10]:
create_model_weights_file(model,100)

Custom class for inference on encrypted data


In [11]:
class Model:
    def __init__(self,n_features,n_layers,n_layer_neurons,layer_activations,layer_weight_files):
        self.n_features=n_features
        self.n_layers=n_layers
        self.n_layer_neurons=n_layer_neurons
        self.layer_activations=layer_activations
        self.weights=[]
        self.biases=[]
        for wt_file in layer_weight_files:
            with open(wt_file, 'rb') as f:
              self.weights.append(np.load(f).tolist())
              self.biases.append(np.load(f).tolist())
    def __matmul(self,a,b):
        ## Input: a to be 2-D (n_layer_neurons[layer_no],n_features)and b to be 1-D list
        # Output : a 1-D list 
        output=[]
        for i in range(len(a)):
            res=0
            for j in range(len(b)):
                res=a[i][j]*b[j]+res
            output.append(res)
        return output
    
    def __matadd(self,a,b):
        ## Input: a and b to be 1-D list
        # Output : a 1-D list 
        ## For memory saving b is modified in calculations so be careful about order of inputs
        
        for i in range(len(a)):
            b[i]=b[i]+a[i]
        return b  
    
    def infer(self,x,encrypted=True):
        for i in range(self.n_layers):
            
            new_x=self.__matmul(self.weights[i],x)
            new_x=self.__matadd(self.biases[i],new_x)
            
            if(self.layer_activations[i]=='relu'):
               
                x=[t**2+t for t in new_x]
                # if(encrypted==True):
                #     temp=copy.deepcopy(new_x)
                #     for t in temp:
                #         print(private_key.decrypt(t))
                #     temp=copy.deepcopy(x)
                #     for t in temp:
                #         print(private_key.decrypt(t))
                if(encrypted==True):
                    x=[t**2+t for t in new_x]
                # else:
                #     x=[max(0,t) for t in new_x]
            else:  
                x=new_x  
        return x[0]      
       
            
        
        
        
        

In [12]:
model=Model(30,2,[2,1],['none','none'],['layer0.npy','layer1.npy'])

Test out accuracy on Encrypted and Unencrypted test data

In [13]:
normal_acc=0
encrypted_acc=0
limit=len(X_test)
for i in range(limit):
    x=X_test[i].tolist()
    x_encrypted=[public_key.encrypt(t) for t in x]
    normal_res=model.infer(x,False)
    encrypted_res=private_key.decrypt(model.infer(x_encrypted))
    #print('Unencryted Inference:'+str(normal_res)+" Encrypted Inference:"+str(encrypted_res))
    if (normal_res>0 and y_test[i]==1) or (normal_res<=0 and y_test[i]==0):
        normal_acc+=1
    if (encrypted_res>0 and y_test[i]==1) or (encrypted_res<=0 and y_test[i]==0):
        encrypted_acc+=1
print('Accuracy on unencrypted Test Data:'+str(normal_acc/limit))
print('Accuracy on encrypted test data:'+str(encrypted_acc/limit))
  

Accuracy on unencrypted Test Data:0.9300699300699301
Accuracy on encrypted test data:0.9300699300699301
